In [4]:
import pandas as pd
import numpy as np

# Handling Missing Data

In [2]:
#Pandas chose to use sentinels for missing data, and
#further chose to use two already-existing Python null values: the special floating-
#point NaN value, and the Python None object. This choice has some side effects, as we
#will see, but in practice ends up being a good compromise in most cases of interest.

# None: Pythonic missing data

In [3]:
#The first sentinel value used by Pandas is None , a Python singleton object that is often
#used for missing data in Python code. Because None is a Python object, it cannot be
#used in any arbitrary NumPy/Pandas array, but only in arrays with data type
#'object' (i.e., arrays of Python objects)

In [5]:
vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

In [6]:
#This dtype=object means that the best common type representation NumPy could
#infer for the contents of the array is that they are Python objects. While this kind of
#object array is useful for some purposes, any operations on the data will be done at
#the Python level, with much more overhead than the typically fast operations seen for
#arrays with native types

In [7]:
#The use of Python objects in an array also means that if you perform aggregations
#like sum() or min() across an array with a None value, you will generally get an error

In [8]:
#This reflects the fact that addition between an integer and None is undefined.

# NaN: Missing numerical data

In [9]:
#The other missing data representation, NaN (acronym for Not a Number), is different;
#it is a special floating-point value recognized by all systems that use the standard
#IEEE floating-point representation

In [10]:
a=np.array([1,np.nan,3,4])

In [11]:
a

array([ 1., nan,  3.,  4.])

In [12]:
a.dtype

dtype('float64')

In [13]:
#Notice that NumPy chose a native floating-point type for this array: this means that
#unlike the object array from before, this array supports fast operations pushed into
#compiled code. You should be aware that NaN is a bit like a data virus—it infects any
#other object it touches. Regardless of the operation, the result of arithmetic with NaN
#will be another NaN

In [14]:
1+np.nan

nan

In [15]:
0-np.nan

nan

In [17]:
#Note that this means that aggregates over the values are well defined (i.e., they don’t
#result in an error) but not always useful

In [18]:
a.sum()

nan

In [19]:
a.min()

nan

In [20]:
a.max()

nan

In [21]:
#NumPy does provide some special aggregations that will ignore these missing values

In [22]:
np.nansum(a)

8.0

In [23]:
np.nanmin(a)

1.0

In [24]:
np.nanmax(a)

4.0

In [25]:
np.nanmean(a)

2.6666666666666665

In [26]:
#Keep in mind that NaN is specifically a floating-point value; there is no equivalent
#NaN value for integers, strings, or other types

# NaN and None in Pandas

In [27]:
#NaN and None both have their place, and Pandas is built to handle the two of them
#nearly interchangeably, converting between them where appropriate

In [28]:
s=pd.Series([1,np.nan,2,None])

In [30]:
s

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

In [31]:
#For types that don’t have an available sentinel value, Pandas automatically type-casts
#when NA values are present. For example, if we set a value in an integer array to
#np.nan , it will automatically be upcast to a floating-point type to accommodate the
#NA

In [32]:
x = pd.Series(range(2), dtype=int)
x

0    0
1    1
dtype: int64

In [33]:
x[0] = None
x

0    NaN
1    1.0
dtype: float64

In [34]:
#Keep in mind that in Pandas, string data is always stored with an object dtype.

# Operating on Null Values

In [35]:
#As we have seen, Pandas treats None and NaN as essentially interchangeable for indi‐
#cating missing or null values. To facilitate this convention, there are several useful
#methods for detecting, removing, and replacing null values in Pandas data structures.
#They are:

In [36]:
#isnull()
#Generate a Boolean mask indicating missing values
#notnull()
#Opposite of isnull()
#dropna()
#Return a filtered version of the data
#fillna()
#Return a copy of the data with missing values filled or imputed

# Detecting null values

In [37]:
data = pd.Series([1, np.nan, 'hello', None])

In [39]:
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

In [40]:
data[data.notnull()]

0        1
2    hello
dtype: object

# Dropping null values

In [41]:
data

0        1
1      NaN
2    hello
3     None
dtype: object

In [42]:
data.dropna()

0        1
2    hello
dtype: object

In [43]:
df = pd.DataFrame([[1,
np.nan, 2],
[2,
3,
5],
[np.nan, 4,
6]])

In [44]:
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [45]:
df.dropna() #axis=0 (by default)

,0,1,2
1,2.0,3.0,5


In [46]:
df.dropna(axis=1)

,2
0,2
1,5
2,6


In [47]:
#But this drops some good data as well; you might rather be interested in dropping
#rows or columns with all NA values, or a majority of NA values. This can be specified
#through the how or thresh parameters, which allow fine control of the number of
#nulls to allow through.
#The default is how='any' , such that any row or column (depending on the axis key‐
#word) containing a null value will be dropped. You can also specify how='all' , which
#will only drop rows/columns that are all null values

In [48]:
df[3] = np.nan

In [49]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [50]:
df.dropna(axis=1,how='all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [51]:
#For finer-grained control, the thresh parameter lets you specify a minimum number
#of non-null values for the row/column to be kept

In [52]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [53]:
df.dropna(axis=0,thresh=3)

,0,1,2,3
1,2.0,3.0,5,NaN


In [54]:
#Here the first and last row have been dropped, because they contain only two non-
#null values.

# Filling null values

In [55]:
#Sometimes rather than dropping NA values, you’d rather replace them with a valid
#value. This value might be a single number like zero, or it might be some sort of
#imputation or interpolation from the good values. You could do this in-place using
#the isnull() method as a mask, but because it is such a common operation Pandas
#provides the fillna() method, which returns a copy of the array with the null values
#replaced.

In [56]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [57]:
#We can fill NA entries with a single value, such as zero

In [58]:
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

In [59]:
#We can specify a forward-fill to propagate the previous value forward

In [60]:
data.fillna(method="ffill")

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

In [61]:
#Or we can specify a back-fill to propagate the next values backward

In [62]:
data.fillna(method="bfill")

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

In [63]:
#For DataFrame s, the options are similar, but we can also specify an axis along which
#the fills take place

In [64]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [65]:
df.fillna(axis=1,method="ffill")

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0


In [66]:
df.fillna(axis=1,method="bfill")

,0,1,2,3
0,1.0,2.0,2.0,NaN
1,2.0,3.0,5.0,NaN
2,4.0,4.0,6.0,NaN


In [67]:
df.fillna(axis=0,method="ffill")

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,2.0,4.0,6,NaN


In [68]:
df.fillna(axis=0,method="bfill")

,0,1,2,3
0,1.0,3.0,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


# Thank You